In [ ]:
!nvidia-smi

Sat Dec  3 07:55:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Input dataset

Dataset from :  https://www.kaggle.com/datasets/francismon/curated-covid19-chest-xray-dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile ('/content/drive/MyDrive/Dataset Noor Muhamad Rizki/X-ray dataset.zip')
zip_ref.extractall()
zip_ref.close()

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import itertools
import pandas as pd
import matplotlib.image as mpimg
import os
import random
import datetime


from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.metrics import confusion_matrix
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

tf.__version__

'2.9.2'

In [ ]:
def plot_loss_curves(history):

  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

In [ ]:
def load_and_prep_image(filename, img_shape=224, scale=True):
  img = tf.io.read_file(filename)

  img = tf.io.decode_image(img)

  img = tf.image.resize(img, [img_shape, img_shape])
  if scale:

    return img/255.
  else:
    return img

In [ ]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

# Our function needs a different name to sklearn's plot_confusion_matrix
def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15, norm=False, savefig=False): 
 

  # Create the confustion matrix
  cm = confusion_matrix(y_true, y_pred)
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
  n_classes = cm.shape[0] # find the number of classes we're dealing with

  # Plot the figure and make it pretty
  fig, ax = plt.subplots(figsize=figsize)
  cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
  fig.colorbar(cax)

  # Are there a list of classes?
  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])
  
  # Label the axes
  ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes), 
         xticklabels=labels, # axes will labeled with class names (if they exist) or ints
         yticklabels=labels)
  
  # Make x-axis labels appear on bottom
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  ### Added: Rotate xticks for readability & increase font size (required due to such a large confusion matrix)
  plt.xticks(rotation=70, fontsize=text_size)
  plt.yticks(fontsize=text_size)

  # Set the threshold for different colors
  threshold = (cm.max() + cm.min()) / 2.

  # Plot the text on each cell
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    if norm:
      plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)
    else:
      plt.text(j, i, f"{cm[i, j]}",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)

  if savefig:
    fig.savefig("confusion_matrix.png")

In [ ]:
def compare_historys(original_history, new_history, initial_epochs=5):

    
    # Get original history measurements
    acc = original_history.history["accuracy"]
    loss = original_history.history["loss"]

    val_acc = original_history.history["val_accuracy"]
    val_loss = original_history.history["val_loss"]

    # Combine original history with new history
    total_acc = acc + new_history.history["accuracy"]
    total_loss = loss + new_history.history["loss"]

    total_val_acc = val_acc + new_history.history["val_accuracy"]
    total_val_loss = val_loss + new_history.history["val_loss"]

    # Make plots
    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(total_acc, label='Training Accuracy')
    plt.plot(total_val_acc, label='Validation Accuracy')
    plt.plot([initial_epochs-1, initial_epochs-1],
              plt.ylim(), label='Start Fine Tuning') # reshift plot around epochs
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(total_loss, label='Training Loss')
    plt.plot(total_val_loss, label='Validation Loss')
    plt.plot([initial_epochs-1, initial_epochs-1],
              plt.ylim(), label='Start Fine Tuning') # reshift plot around epochs
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

In [ ]:
def tensorboard_callback(dir_name, experiment_name):

  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

In [ ]:
import os

for dirpath, filenames, dirnames in os.walk('dataset'):
  print (f"Tersedia {len(dirnames)} direktori dan  {len(filenames)} citra pada  {dirpath}")

Tersedia 0 direktori dan  2 citra pada  dataset
Tersedia 0 direktori dan  3 citra pada  dataset/train
Tersedia 2616 direktori dan  0 citra pada  dataset/train/0_Normal
Tersedia 3726 direktori dan  0 citra pada  dataset/train/2_Pneumonia
Tersedia 1025 direktori dan  0 citra pada  dataset/train/1_Covid19
Tersedia 0 direktori dan  3 citra pada  dataset/validation
Tersedia 654 direktori dan  0 citra pada  dataset/validation/0_Normal
Tersedia 931 direktori dan  0 citra pada  dataset/validation/2_Pneumonia
Tersedia 256 direktori dan  0 citra pada  dataset/validation/1_Covid19


# Preprocessing Data

In [ ]:
data_direktori = os.path.join ('/content/dataset')
train_dir = os.path.join ('/content/dataset/train')
validation_dir = os.path.join ('/content/dataset/validation')


In [ ]:
IMG_SIZE = (224, 224) 
train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                            image_size=IMG_SIZE,
                                                                            label_mode="categorical", # what type are the labels?
                                                                            batch_size=32) # batch_size is 32 by default, this is generally a good number
validation_data = tf.keras.preprocessing.image_dataset_from_directory(validation_dir,
                                                                           image_size=IMG_SIZE,
                                                                           label_mode="categorical",
                                                               shuffle = False)

In [ ]:
class_names = train_data.class_names

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_accuracy')>0.9814):
      print("\nReached 9815% accuracy so cancelling training!")
      self.model.stop_training = True

maks = myCallback()

#Make Model

In [ ]:
input_shape = (224, 224, 3)
base_model = tf.keras.applications.MobileNetV3Small(include_top=False, weights='imagenet', input_shape=(224,224,3))
base_model.trainable = False

x = base_model.output
x = layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)

x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(3, activation="softmax", name="output_layer")(x)


model= keras.Model(base_model.input, outputs)

model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

# Fit the model
history= model.fit(train_dir,
                    epochs=50,
                    validation_data=validation_dir,
                    callbacks=[maks])

In [ ]:
train_result = model.evaluate(train_dir)
test_result = model.evaluate(validation_dir)

df = pd.DataFrame(zip(train_result,test_result),columns=['Train','Val'],index=['Loss','Acc'])
df

In [ ]:
plot_loss_curves(history)

#Melakukan Fine Tunning pada model tanpa Augmentasi


In [ ]:
# Unfreeze all of the layers in the base model
base_model.trainable = True

# Refreeze every layer except for the last 5
for layer in base_model.layers[:-23]:
  layer.trainable = False

for layer_number, layer in enumerate(base_model.layers):
  print(layer_number, layer.name, layer.trainable)

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4), # 10x lower learning rate than default
              metrics=['accuracy'])

fine_tune_epochs = 100 # model has already done 10 epochs, this is the total number of epochs we're after (5+5=10)

history_fine = model.fit(train_data,
                                                    epochs=fine_tune_epochs,
                                                     validation_data=validation_data,
                                                     initial_epoch=history.epoch[-1], # start from previous last epoch
                                                     callbacks=[maks])

In [ ]:
train_result = model.evaluate(train_data)
test_result = model.evaluate(validation_data)

fine = pd.DataFrame(zip(train_result,test_result),columns=['Train','Val'],index=['Loss','Acc'])
fine_df

In [ ]:
compare_historys(original_history=history,
                 new_history=history_fine,
                 initial_epochs=50)

#Evaluation model

## Confussion matrix

In [ ]:
pred_probs= model.predict(validation_data, verbose=1)

In [ ]:
pred_classes = pred_probs.argmax(axis=1)
y_labels = []
for images, labels in validation_data.unbatch(): # unbatch the test data and get images and labels
  y_labels.append(labels.numpy().argmax()) # append the index which has the largest value (labels are one-hot)

In [ ]:
from sklearn.metrics import accuracy_score
sklearn_accuracy = accuracy_score(y_labels, pred_classes)
sklearn_accuracy

In [ ]:
make_confusion_matrix(y_true=y_labels,
                      y_pred=pred_classes,
                      classes=class_names,
                      figsize=(10, 10),
                      text_size=20,
                      norm=False)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_labels, pred_classes))

## Find Most Wrong images

In [ ]:
filepaths = []
for filepath in data_uji.list_files("/content/dataset/validation/*/*.jpg", 
                                     shuffle=False):
  filepaths.append(filepath.numpy())
filepaths[:10]

In [ ]:
import pandas as pd
pred_df = pd.DataFrame({"img_path": filepaths,
                        "y_true": y_labels,
                        "y_pred": pred_classes,
                        "pred_conf": pred_probs.max(axis=1), # get the maximum prediction probability value
                        "y_true_classname": [class_names[i] for i in y_labels],
                        "y_pred_classname": [class_names[i] for i in pred_classes]}) 
pred_df.head()

In [ ]:
pred_df["pred_correct"] = pred_df["y_true"] == pred_df["y_pred"]
pred_df.head()

In [ ]:
top_100_wrong = pred_df[pred_df["pred_correct"] == False].sort_values("pred_conf", ascending=False)[:100]
top_100_wrong.head(50)

In [ ]:
import os
import random

plt.figure(figsize=(4, 4))

  # Choose a random image from a random class 
filepath = '/content/dataset/validation/2_Pneumonia/Pneumonia-Viral (1363).jpg'
class_name = class_names[2]

  # Load the image and make predictions
img = load_and_prep_image(filepath, scale=False) # don't scale images for EfficientNet predictions
pred_prob = best_model.predict(tf.expand_dims(img, axis=0)) # model accepts tensors of shape [None, 224, 224, 3]
pred_class = class_names[pred_prob.argmax()] # find the predicted class 

  # Plot the image(s)
plt.imshow(img/255.)
if class_name == pred_class: # Change the color of text based on whether prediction is right or wrong
  title_color = "g"
else:
  title_color = "r"

plt.title(f"citra asli : {class_name} \n prediksi: {pred_class}: {100*pred_prob.max():.2f}%", c=title_color, fontsize=12,fontweight=20)
plt.axis(False);

## Grad-Cam Visualization

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

IMAGE_PATH = filepath
LAYER_NAME = 'Conv_1'
CLASS_INDEX = 2

img = tf.keras.preprocessing.image.load_img(IMAGE_PATH, target_size=(224, 224))
img = tf.keras.preprocessing.image.img_to_array(img)

# Load initial model
model_grad = model_2

# Create a graph that outputs target convolution and output
grad_model = tf.keras.models.Model([model_grad.inputs], [model_grad.get_layer(LAYER_NAME).output, model_grad.output])

# Get the score for target class
with tf.GradientTape() as tape:
    conv_outputs, predictions = grad_model(np.array([img]))
    loss = predictions[:, CLASS_INDEX]

# Extract filters and gradients
output = conv_outputs[0]
grads = tape.gradient(loss, conv_outputs)[0]

# Average gradients spatially
weights = tf.reduce_mean(grads, axis=(0, 1))

# Build a ponderated map of filters according to gradients importance
cam = np.ones(output.shape[0:2], dtype=np.float32)

for index, w in enumerate(weights):
    cam += w * output[:, :, index]

# Heatmap visualization
cam = cv2.resize(cam.numpy(), (224, 224))
cam = np.maximum(cam, 0)
heatmap = (cam - cam.min()) / (cam.max() - cam.min())

cam = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)

output_image = cv2.addWeighted(cv2.cvtColor(img.astype('uint8'), cv2.COLOR_RGB2BGR), 0.5, cam, 1, 0)

plt.figure(figsize=(10, 10))
plt.imshow(output_image)
plt.title(f"real image : {class_name} \n prediction: {pred_class}: {100*pred_prob.max():.2f}%", c=title_color, fontsize=18,fontweight=20)
plt.axis(False);